In [2]:
# Imports
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from copy import deepcopy
import gymnasium as gym

import os

from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time

os.environ["SDL_VIDEODRIVER"] = "dummy"


def eval_agent(agent, env, n_sim=5):
    """
    ** Solution **

    Monte Carlo evaluation of DQN agent.

    Repeat n_sim times:
        * Run the DQN policy until the environment reaches a terminal state (= one episode)
        * Compute the sum of rewards in this episode
        * Store the sum of rewards in the episode_rewards array.
    """
    env_copy = deepcopy(env)
    episode_rewards = np.zeros(n_sim)
    for i in range(n_sim):
        state, _ = env_copy.reset()
        reward_sum = 0
        done = False
        while not done:
            action = agent.get_action(state, 0)
            state, reward, terminated, truncated, _ = env_copy.step(action)
            reward_sum += reward
            done = terminated or truncated
        episode_rewards[i] = reward_sum
    return episode_rewards


class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, state, action, reward, terminated, next_state):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = (state, action, reward, terminated, next_state)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.choices(self.memory, k=batch_size)

    def __len__(self):
        return len(self.memory)


class Net(nn.Module):
    """
    Basic neural net.
    """

    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
        )

    def forward(self, x):
        # return self.net(x)
        x = x.view(x.size(0), -1)
        return self.net(x)


class DQN:
    def __init__(
        self,
        env,
        action_space,
        observation_space,
        gamma,
        batch_size,
        buffer_capacity,
        update_target_every,
        epsilon_start,
        decrease_epsilon_factor,
        epsilon_min,
        learning_rate,
    ):
        self.env = env
        self.action_space = action_space
        self.observation_space = observation_space
        self.gamma = gamma

        self.batch_size = batch_size
        self.buffer_capacity = buffer_capacity
        self.update_target_every = update_target_every

        self.epsilon_start = epsilon_start
        self.decrease_epsilon_factor = (
            decrease_epsilon_factor  # larger -> more exploration
        )
        self.epsilon_min = epsilon_min

        self.learning_rate = learning_rate

        self.reset()

    def update(self, state, action, reward, terminated, next_state):
        """
        ** SOLUTION **
        """
        # Convert numpy arrays or lists to tensors and ensure they are floats
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        next_state_tensor = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0)
        action_tensor = torch.tensor([[action]], dtype=torch.int64)
        reward_tensor = torch.tensor([reward], dtype=torch.float32)
        terminated_tensor = torch.tensor([terminated], dtype=torch.float32)

        # Store transition in the replay buffer
        self.buffer.push(
            state_tensor,
            action_tensor,
            reward_tensor,
            terminated_tensor,
            next_state_tensor,
        )

        # # add data to replay buffer
        # self.buffer.push(
        #     torch.tensor(state).unsqueeze(0),
        #     torch.tensor([[action]], dtype=torch.int64),
        #     torch.tensor([reward]),
        #     torch.tensor([terminated], dtype=torch.int64),
        #     torch.tensor(next_state, dtype=torch.float).unsqueeze(0),
        # )

        if len(self.buffer) < self.batch_size:
            return np.inf

        # get batch
        transitions = self.buffer.sample(self.batch_size)

        # Compute loss - TO BE IMPLEMENTED!
        # Hint: use the gather method from torch.

        state_batch, action_batch, reward_batch, terminated_batch, next_state_batch = (
            tuple([torch.cat(data) for data in zip(*transitions)])
        )

        values = self.q_net.forward(state_batch).gather(1, action_batch)

        # Compute the ideal Q values
        with torch.no_grad():
            next_state_values = (1 - terminated_batch) * self.target_net(
                next_state_batch
            ).max(1)[0]
            targets = next_state_values * self.gamma + reward_batch

        loss = self.loss_function(values, targets.unsqueeze(1))

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_value_(self.q_net.parameters(), 100)
        self.optimizer.step()

        if not ((self.n_steps + 1) % self.update_target_every):
            self.target_net.load_state_dict(self.q_net.state_dict())

        self.decrease_epsilon()

        self.n_steps += 1
        if terminated:
            self.n_eps += 1

        return loss.detach().numpy()

    def get_action(self, state, epsilon=None):
        """
        Return action according to an epsilon-greedy exploration policy
        """
        if epsilon is None:
            epsilon = self.epsilon

        if np.random.rand() < epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.get_q(state))

    def get_q(self, state):
        """
        Compute Q function for a states
        """
        state_tensor = torch.tensor(state, dtype=torch.float).unsqueeze(0)
        with torch.no_grad():
            output = self.q_net.forward(state_tensor)  # shape (1,  n_actions)
        return output.numpy()[0]  # shape  (n_actions)

    def decrease_epsilon(self):
        self.epsilon = self.epsilon_min + (self.epsilon_start - self.epsilon_min) * (
            np.exp(-1.0 * self.n_eps / self.decrease_epsilon_factor)
        )

    def reset(self):
        hidden_size = 128

        # obs_size = self.observation_space.shape[0]
        obs_size = np.prod(self.env.observation_space.shape)
        n_actions = self.action_space.n

        self.buffer = ReplayBuffer(self.buffer_capacity)
        self.q_net = Net(obs_size, hidden_size, n_actions)
        self.target_net = Net(obs_size, hidden_size, n_actions)

        self.loss_function = nn.MSELoss()
        self.optimizer = optim.Adam(
            params=self.q_net.parameters(), lr=self.learning_rate
        )

        self.epsilon = self.epsilon_start
        self.n_steps = 0
        self.n_eps = 0


class RandomAgent:
    def __init__(self, observation_space, action_space):
        self.action_space = action_space
        return

    def get_action(self, state, *args):
        return self.action_space.sample()

    def update(self, *data):
        pass


def run_one_episode(env, agent, display=True):
    display_env = deepcopy(env)
    done = False
    state, _ = display_env.reset()

    rewards = 0

    while not done:
        action = agent.get_action(state, 0)
        state, reward, done, _, _ = display_env.step(action)
        rewards += reward
        if display:
            clear_output(wait=True)
            plt.imshow(display_env.render())
            plt.show()
    if display:
        display_env.close()
    print(f"Episode length {rewards}")


def run_episodes_for_a_minute(env, agent, display=True):
    start_time = time.time()
    episodes = 0
    total_rewards = 0

    while time.time() - start_time < 10:  # Exécuter pendant environ une minute
        state, _ = env.reset()
        done = False
        episode_rewards = 0

        while not done:
            action = agent.get_action(state, 0)
            state, reward, done, _, _ = env.step(action)
            episode_rewards += reward
            if display:
                clear_output(wait=True)
                plt.imshow(env.render())
                plt.show()

        episodes += 1
        total_rewards += episode_rewards
        print(f"Episode {episodes} reward: {episode_rewards}")

    print(f"Nombre total d'épisodes exécutés en 1 minute : {episodes}")
    print(f"Récompense moyenne sur les épisodes : {total_rewards / episodes}")


# env = env = gym.make("CartPole-v1", render_mode="rgb_array")
# agent = RandomAgent(env.observation_space, env.action_space)

# # Exécuter la fonction pour des épisodes répétés pendant environ une minute
# run_episodes_for_a_minute(env, agent)

# # run_one_episode(env, agent, display=True)
# print(f"Average over 5 runs : {np.mean(eval_agent(agent, env))}")

In [3]:
def train(env, agent, N_episodes, eval_every=10, reward_threshold=300):
    total_time = 0
    state, _ = env.reset()
    losses = []
    for ep in range(N_episodes):
        done = False
        state, _ = env.reset()
        while not done:
            action = agent.get_action(state)

            next_state, reward, terminated, truncated, _ = env.step(action)
            loss_val = agent.update(state, action, reward, terminated, next_state)

            state = next_state
            losses.append(loss_val)

            done = terminated or truncated
            total_time += 1

        if (ep + 1) % eval_every == 0:
            rewards = eval_agent(agent, env)
            print("episode =", ep + 1, ", reward = ", np.mean(rewards))
            if np.mean(rewards) >= reward_threshold:
                break

    return losses

In [4]:
# Adjusted environment initialization for "highway-fast-v0"
import gymnasium as gym
from gymnasium import spaces
from config import config

# Configure and create the environment
env = gym.make("highway-fast-v0", render_mode="rgb_array", config=config)
# env.unwrapped.configure(config)
action_space = env.action_space
observation_space = env.observation_space

# Update the DQN Agent initialization with the new action and observation spaces
# Note: Ensure that observation dimensions are correctly handled within your DQN architecture.
# This might require adjustments depending on how the "OccupancyGrid" observations are structured.

# Hyperparameters might need adjustment based on the new environment dynamics.
gamma = 0.99
batch_size = 128
buffer_capacity = 10_000
update_target_every = 32
epsilon_start = 0.9
decrease_epsilon_factor = 1000
epsilon_min = 0.05
learning_rate = 1e-5

hidden_size = 128
n_actions = env.action_space.n

# When you instantiate the DQN agent:
agent = DQN(
    env,
    action_space,
    observation_space,
    gamma,
    batch_size,
    buffer_capacity,
    update_target_every,
    epsilon_start,
    decrease_epsilon_factor,
    epsilon_min,
    learning_rate,
)

# Training might need adjustments, especially evaluation metrics and thresholding for success.
N_episodes = 300

# Proceed with the adjusted training function
# Ensure that your training and evaluation routines properly handle the updated environment observations and actions.
losses = train(env, agent, N_episodes)
plt.plot(losses)

AttributeError: 'Discrete' object has no attribute 'observation_space'

In [63]:
# Evaluate the final policy
rewards = eval_agent(agent, env, 20)
print("")
print("mean reward after training = ", np.mean(rewards))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x448 and 25x128)

In [59]:
def visualize_agent_performance(env, agent, delay=0.1):
    """
    Visualizes the agent's performance in the environment.

    :param env: The gym environment.
    :param agent: The trained agent.
    :param delay: Time delay between frames in seconds for better visualization.
    """
    # Reset the environment and get the initial state
    state, _ = env.reset()

    # Initialize done flag
    done = False

    # Loop until the episode is finished
    while True:
        done = truncated = False
        obs, info = env.reset()
        while not (done or truncated):
            action = agent.get_action(state, 0)
            obs, reward, done, truncated, info = env.step(action)
            env.render()


In [61]:
%matplotlib inline
visualize_agent_performance(env, agent)

KeyboardInterrupt: 